In [ ]:
project = 'saga-nvdb-prod-vlmh'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter ut ID, startdato, navn, åpningsår og offisiell lengde for aktive tunneler i Trøndelag, sortert på lengde, synkende.

In [ ]:
query = f"""
SELECT
  id,
  metadata.startdato,
  egenskaper.navn,
  egenskaper.aapningsaar,
  egenskaper.lengde_offisiell
FROM
  `{project}.standardized.vegobjekter_tunnel`
WHERE
  metadata.sluttdato IS NULL
  AND 50 IN UNNEST(lokasjon.fylker)
ORDER BY
  lengde_offisiell DESC
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen henter ID, kortform, underlagte tunnelløp, startdato, navn og lengde for aktive undersjøiske tunneler, sortert på navn.

In [ ]:
query = f"""
SELECT
  id,
  (SELECT kortform FROM UNNEST(lokasjon.vegsystemreferanser)) kortform,
  (SELECT vegobjekter FROM UNNEST(relasjoner.barn) WHERE type.id = 67) tunnellop,
  metadata.startdato, 
  egenskaper.navn,
  egenskaper.lengde_offisiell
FROM
  `{project}.standardized.vegobjekter_tunnel`
WHERE
  metadata.sluttdato IS NULL
  AND egenskaper.undersjoisk = 'Ja'
ORDER BY
  navn
"""

print(query)

client.query(query).to_dataframe()